# day 11

https://adventofcode.com/11/day/11

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day11.txt')

LOGGER = logging.getLogger('day11')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
import networkx as nx

def parse_data(data: str) -> tuple[nx.Graph, list[complex]]:
    galaxies = []
    g = nx.Graph()
    for (i, line) in enumerate(data.strip().split('\n')):
        for (j, c) in enumerate(line.strip()):
            # real increases downward
            # imaginary increases rightward
            idx = i + j * 1j
            if c == '#':
                galaxies.append(idx)
            for dir in [1, 1j]:
                g.add_edge(idx, idx + dir)
            W = j + 1
        H = i + 1

    empty_rows = set(range(H)).difference(x.real for x in galaxies)
    empty_cols = set(range(W)).difference(x.imag for x in galaxies)
    
    return g, galaxies, H, W, empty_rows, empty_cols

In [ ]:
g, galaxies, H, W, empty_rows, empty_cols = parse_data(test_data)
galaxies

In [ ]:
H, W

In [ ]:
empty_rows

In [ ]:
empty_cols

In [ ]:
def build_wt_func(empty_rows: set[int], empty_cols: set[int]):
    def wt_func(u: complex, v: complex, d: dict) -> int:
        if (u.real in empty_rows) and (v.real == u.real + 1):
            return 2.0
        elif (v.real in empty_rows) and (u.real == v.real + 1):
            return 2.0
        elif (u.imag in empty_cols) and (v.imag == u.imag + 1):
            return 2.0
        elif (v.imag in empty_cols) and (u.imag == v.imag + 1):
            return 2.0
        else:
            return 1.0
    return wt_func

In [ ]:
wt_func = build_wt_func(empty_rows=empty_rows, empty_cols=empty_cols)
assert wt_func(2j, 3j, {}) == 2.0
assert wt_func(3j, 2j, {}) == 2.0
assert wt_func(3, 4, {}) == 2.0
assert wt_func(4, 3, {}) == 2.0

In [ ]:
assert nx.shortest_path_length(g, source=3j, target=8 + 7j, weight=wt_func) == 15.0
assert nx.shortest_path_length(g, source=8 + 7j, target=3j, weight=wt_func) == 15.0
assert nx.shortest_path_length(g, source=5 + 1j, target=9+4j, weight=wt_func) == 9.0
assert nx.shortest_path_length(g, source=9 + 4j, target=5 + 1j, weight=wt_func) == 9.0

#### function def

In [ ]:
def get_path_lens(g, node_list, wt_func):
    path_lens = {}
    for (i, n0) in enumerate(node_list[:-1], start=1):
        LOGGER.info(f"i = {i}")
        for (j, n1) in enumerate(node_list[i:], start=i + 1):
            LOGGER.info(f"  j = {j}")
            path_lens[i, j] = nx.shortest_path_length(g, source=n0, target=n1, weight=wt_func)
    return path_lens


def get_path_lens_2(g, node_list, empty_rows, empty_cols, exp_coef=2):
    path_lens = {}
    for (i, n0) in enumerate(node_list[:-1], start=1):
        for (j, n1) in enumerate(node_list[i:], start=i + 1):
            real_diff = abs(n1.real - n0.real) + (exp_coef - 1) * len(empty_rows.intersection(range(int(min(n0.real, n1.real)), int(max(n0.real, n1.real)))))
            imag_diff = abs(n1.imag - n0.imag) + (exp_coef - 1) * len(empty_cols.intersection(range(int(min(n0.imag, n1.imag)), int(max(n0.imag, n1.imag))))) 
            path_lens[i, j] = real_diff + imag_diff
    return path_lens

In [ ]:
g, galaxies, H, W, empty_rows, empty_cols = parse_data(data=test_data)
wt_func = build_wt_func(empty_rows=empty_rows, empty_cols=empty_cols)
path_lens = get_path_lens(g=g, node_list=galaxies, wt_func=wt_func)
path_lens_2 = get_path_lens_2(g=g, node_list=galaxies, empty_rows=empty_rows, empty_cols=empty_cols)
assert path_lens == path_lens_2

In [ ]:
def q_1(data):
    g, galaxies, H, W, empty_rows, empty_cols = parse_data(data=data)
    # wt_func = build_wt_func(empty_rows=empty_rows, empty_cols=empty_cols)
    # path_lens = get_path_lens(g=g, node_list=galaxies, wt_func=wt_func)
    path_lens = get_path_lens_2(g=g, node_list=galaxies, empty_rows=empty_rows, empty_cols=empty_cols)
    return sum(path_lens.values())

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 374
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data, exp_coef=2.0):
    g, galaxies, H, W, empty_rows, empty_cols = parse_data(data=data)
    # wt_func = build_wt_func(empty_rows=empty_rows, empty_cols=empty_cols)
    # path_lens = get_path_lens(g=g, node_list=galaxies, wt_func=wt_func)
    path_lens = get_path_lens_2(g=g, node_list=galaxies, empty_rows=empty_rows, empty_cols=empty_cols, exp_coef=exp_coef)
    return sum(path_lens.values())

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data, 10) == 1030
    assert q_2(test_data, 100) == 8410
    LOGGER.setLevel(logging.INFO)

In [ ]:
q_2(test_data, 10)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data(), 1_000_000)

fin